# Logistic Regression Documentation Code Example

In [1]:
import findspark
findspark.init('/home/kajili/spark-2.4.5-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('log-reg').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [6]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [7]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



### Note the data is binary classification based (label is either 1 or 0)

In [9]:
log_reg_model = LogisticRegression()

In [10]:
fitted_logreg = log_reg_model.fit(my_data)

In [11]:
log_summary = fitted_logreg.summary

In [12]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [16]:
log_summary.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



# Now split into train and test sets

In [17]:
train_logreg_data, test_logreg_data = my_data.randomSplit([0.7,0.3])

In [22]:
logreg_obj = LogisticRegression()
type(logreg_obj)

pyspark.ml.classification.LogisticRegression

In [23]:
final_logreg_model = logreg_obj.fit(train_logreg_data)

In [24]:
prediction_and_labels = final_logreg_model.evaluate(test_logreg_data)

In [26]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[122,123,124...|[18.9418488687546...|[0.99999999406173...|       0.0|
|  0.0|(692,[123,124,125...|[31.2694826624919...|[0.99999999999997...|       0.0|
|  0.0|(692,[124,125,126...|[31.0920032565615...|[0.99999999999996...|       0.0|
|  0.0|(692,[124,125,126...|[31.6630456786705...|[0.99999999999998...|       0.0|
|  0.0|(692,[124,125,126...|[20.7818529250587...|[0.99999999905690...|       0.0|
|  0.0|(692,[126,127,128...|[21.5835932332872...|[0.99999999957697...|       0.0|
|  0.0|(692,[126,127,128...|[30.9094782513295...|[0.99999999999996...|       0.0|
|  0.0|(692,[126,127,128...|[17.2002393291458...|[0.99999996611316...|       0.0|
|  0.0|(692,[126,127,128...|[22.9765892823145...|[0.99999999989495...|       0.0|
|  0.0|(692,[127

# Now create evaluation object

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [28]:
my_eval_obj = BinaryClassificationEvaluator()

In [29]:
my_final_roc = my_eval_obj.evaluate(prediction_and_labels.predictions)

In [32]:
my_final_roc

1.0